<a href="https://colab.research.google.com/github/arturomf94/pysyft-crash-course/blob/master/notebooks/comprehensive_pysyft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install 'syft[udacity]'

TensorFlow 1.x selected.


# **Intro**

![alt text](https://github.com/arturomf94/pysyft-crash-course/raw/master/docs/openmined.png)

**PySyft** is a tool for privacy-preserving, decentralized deep learning. 

This is a comprehensive, step-by-step guide to the basic concepts in PySyft. Most of the code here is originally from [the tutorials](https://github.com/OpenMined/PySyft/tree/master/examples/tutorials). A detailed explanation of the concepts treated here can be found there, along with extra resources.

# Basics

## Setup

In [0]:
import sys

import torch
from torch.nn import Parameter
import torch.nn as nn
import torch.nn.functional as F

import syft as sy
hook = sy.TorchHook(torch)

torch.tensor([1,2,3,4,5])

Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.2.so'


tensor([1, 2, 3, 4, 5])

## Remote Tensors

In [0]:
x = torch.tensor([1,2,3,4,5])
y = x + x
print(y)

tensor([ 2,  4,  6,  8, 10])


In [0]:
bob = sy.VirtualWorker(hook, id="bob")

In [0]:
x = torch.tensor([1,2,3,4,5])
y = torch.tensor([1,1,1,1,1])

In [0]:
x_ptr = x.send(bob)
y_ptr = y.send(bob)

In [0]:
x_ptr

(Wrapper)>[PointerTensor | me:28120199206 -> bob:62328455535]

In [0]:
bob._objects

{60645563057: tensor([1, 1, 1, 1, 1]), 62328455535: tensor([1, 2, 3, 4, 5])}

In [0]:
z = x_ptr + x_ptr

In [0]:
z

(Wrapper)>[PointerTensor | me:11085720335 -> bob:96064038205]

In [0]:
bob._objects

{60645563057: tensor([1, 1, 1, 1, 1]),
 62328455535: tensor([1, 2, 3, 4, 5]),
 96064038205: tensor([ 2,  4,  6,  8, 10])}

In [0]:
x_ptr

(Wrapper)>[PointerTensor | me:28120199206 -> bob:62328455535]

In [0]:
x_ptr.location

<VirtualWorker id:bob #objects:3>

In [0]:
bob

<VirtualWorker id:bob #objects:3>

In [0]:
bob == x_ptr.location

True

In [0]:
x_ptr.id_at_location

62328455535

In [0]:
x_ptr.owner

<VirtualWorker id:me #objects:0>

In [0]:
x_ptr

(Wrapper)>[PointerTensor | me:28120199206 -> bob:62328455535]

In [0]:
x_ptr.get()

tensor([1, 2, 3, 4, 5])

In [0]:
y_ptr

(Wrapper)>[PointerTensor | me:58353314446 -> bob:60645563057]

In [0]:
y_ptr.get()

tensor([1, 1, 1, 1, 1])

In [0]:
z.get()

tensor([ 2,  4,  6,  8, 10])

In [0]:
bob._objects

{}

## Pointers

In [0]:
x = torch.tensor([1,2,3,4,5]).send(bob)
y = torch.tensor([1,1,1,1,1]).send(bob)

In [0]:
z = x + y

In [0]:
z

(Wrapper)>[PointerTensor | me:37596671072 -> bob:39497819208]

In [0]:
z.get()

tensor([2, 3, 4, 5, 6])

In [0]:
x

(Wrapper)>[PointerTensor | me:33056540189 -> bob:58760922238]

In [0]:
y

(Wrapper)>[PointerTensor | me:63115171859 -> bob:20883767229]

In [0]:
z = torch.add(x,y); z

(Wrapper)>[PointerTensor | me:33701615306 -> bob:89204595672]

In [0]:
z.get()

tensor([2, 3, 4, 5, 6])

In [0]:
x = torch.tensor([1,2,3,4,5.], requires_grad=True).send(bob)
y = torch.tensor([1,1,1,1,1.], requires_grad=True).send(bob)

In [0]:
z = (x + y).sum()

In [0]:
z.backward()

(Wrapper)>[PointerTensor | me:74105025412 -> bob:25312146529]

In [0]:
x = x.get()

In [0]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [0]:
x.grad

tensor([1., 1., 1., 1., 1.])

In [0]:
%reset -f

# Federated Learning

## Intro

In [0]:
import torch
from torch import nn
from torch import optim

In [0]:
# A Toy Dataset
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]])
target = torch.tensor([[0],[0],[1],[1.]])

# A Toy Model
model = nn.Linear(2,1)

def train():
    # Training Logic
    opt = optim.SGD(params=model.parameters(),lr=0.1)
    for iter in range(20):

        # 1) erase previous gradients (if they exist)
        opt.zero_grad()

        # 2) make a prediction
        pred = model(data)

        # 3) calculate how much we missed
        loss = ((pred - target)**2).sum()

        # 4) figure out which weights caused us to miss
        loss.backward()

        # 5) change those weights
        opt.step()

        # 6) print our progress
        print(loss.data)

In [0]:
train()

tensor(0.2956)
tensor(0.1848)
tensor(0.1180)
tensor(0.0756)
tensor(0.0485)
tensor(0.0311)
tensor(0.0199)
tensor(0.0128)
tensor(0.0082)
tensor(0.0053)
tensor(0.0034)
tensor(0.0022)
tensor(0.0014)
tensor(0.0009)
tensor(0.0006)
tensor(0.0004)
tensor(0.0002)
tensor(0.0002)
tensor(0.0001)
tensor(6.7231e-05)


In [0]:
import syft as sy
hook = sy.TorchHook(torch)

In [0]:
# create a couple workers

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")

In [0]:
# A Toy Dataset
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = torch.tensor([[0],[0],[1],[1.]], requires_grad=True)

# get pointers to training data on each worker by
# sending some training data to bob and alice
data_bob = data[0:2]
target_bob = target[0:2]

data_alice = data[2:]
target_alice = target[2:]

# Iniitalize A Toy Model
model = nn.Linear(2,1)

data_bob = data_bob.send(bob)
data_alice = data_alice.send(alice)
target_bob = target_bob.send(bob)
target_alice = target_alice.send(alice)

# organize pointers into a list
datasets = [(data_bob,target_bob),(data_alice,target_alice)]

In [0]:
from syft.federated.floptimizer import Optims
workers = ['bob', 'alice']
optims = Optims(workers, optim=optim.Adam(params=model.parameters(),lr=0.1))

In [0]:
def train():
    # Training Logic
    for iter in range(10):
        
        # NEW) iterate through each worker's dataset
        for data,target in datasets:
            
            # NEW) send model to correct worker
            model.send(data.location)
            
            #Call the optimizer for the worker using get_optim
            opt = optims.get_optim(data.location.id)
            #print(data.location.id)

            # 1) erase previous gradients (if they exist)
            opt.zero_grad()

            # 2) make a prediction
            pred = model(data)

            # 3) calculate how much we missed
            loss = ((pred - target)**2).sum()

            # 4) figure out which weights caused us to miss
            loss.backward()

            # 5) change those weights
            opt.step()
            
            # NEW) get model (with gradients)
            model.get()

            # 6) print our progress
            print(loss.get()) # NEW) slight edit... need to call .get() on loss\
    
# federated averaging

In [0]:
train()

tensor(0.1555, requires_grad=True)
tensor(1.8964, requires_grad=True)
tensor(0.1289, requires_grad=True)
tensor(1.0328, requires_grad=True)
tensor(0.2476, requires_grad=True)
tensor(0.5976, requires_grad=True)
tensor(0.3456, requires_grad=True)
tensor(0.3595, requires_grad=True)
tensor(0.3930, requires_grad=True)
tensor(0.2245, requires_grad=True)
tensor(0.3923, requires_grad=True)
tensor(0.1469, requires_grad=True)
tensor(0.3513, requires_grad=True)
tensor(0.1002, requires_grad=True)
tensor(0.2823, requires_grad=True)
tensor(0.0688, requires_grad=True)
tensor(0.2005, requires_grad=True)
tensor(0.0455, requires_grad=True)
tensor(0.1217, requires_grad=True)
tensor(0.0284, requires_grad=True)


In [0]:
%reset -f

## Aggregation

In [0]:
import torch
import syft as sy
hook = sy.TorchHook(torch)

In [0]:
bob = sy.VirtualWorker(hook, id='bob')
alice = sy.VirtualWorker(hook, id='alice')

In [0]:
# this is a local tensor
x = torch.tensor([1,2,3,4]); x

tensor([1, 2, 3, 4])

In [0]:
# this sends the local tensor to Bob
x_ptr = x.send(bob)

# this is now a pointer
x_ptr

(Wrapper)>[PointerTensor | me:44411603547 -> bob:17315478130]

In [0]:
# now we can SEND THE POINTER to alice!!!
pointer_to_x_ptr = x_ptr.send(alice)

pointer_to_x_ptr

(Wrapper)>[PointerTensor | me:79110793566 -> alice:44411603547]

In [0]:
# As you can see above, Bob still has the actual data (data is always stored in a LocalTensor type). 
bob._objects

{17315478130: tensor([1, 2, 3, 4])}

In [0]:
# and we can use .get() to get x_ptr back from Alice
x_ptr = pointer_to_x_ptr.get()
x_ptr

(Wrapper)>[PointerTensor | me:44411603547 -> bob:17315478130]

In [0]:
# and then we can use x_ptr to get x back from Bob!
x = x_ptr.get()
x

tensor([1, 2, 3, 4])

In [0]:
bob._objects

{}

In [0]:
alice._objects

{}

In [0]:
p2p2x = torch.tensor([1,2,3,4,5]).send(bob).send(alice)

y = p2p2x + p2p2x

In [0]:
bob._objects

{2864754701: tensor([ 2,  4,  6,  8, 10]),
 90281436240: tensor([1, 2, 3, 4, 5])}

In [0]:
alice._objects

{6187838214: (Wrapper)>[PointerTensor | alice:6187838214 -> bob:90281436240],
 22705716963: (Wrapper)>[PointerTensor | alice:22705716963 -> bob:2864754701]}

In [0]:
y.get().get()

tensor([ 2,  4,  6,  8, 10])

In [0]:
bob._objects

{90281436240: tensor([1, 2, 3, 4, 5])}

In [0]:
alice._objects

{6187838214: (Wrapper)>[PointerTensor | alice:6187838214 -> bob:90281436240]}

In [0]:
p2p2x.get().get()

tensor([1, 2, 3, 4, 5])

In [0]:
bob._objects

{}

In [0]:
alice._objects

{}

In [0]:
# x is now a pointer to the data which lives on Bob's machine
x = torch.tensor([1,2,3,4,5]).send(bob)

In [0]:
print('  bob:', bob._objects)
print('alice:',alice._objects)

  bob: {67087745702: tensor([1, 2, 3, 4, 5])}
alice: {}


In [0]:
x = x.move(alice)

In [0]:
print('  bob:', bob._objects)
print('alice:',alice._objects)

  bob: {}
alice: {67087745702: tensor([1, 2, 3, 4, 5])}


In [0]:
x

(Wrapper)>[PointerTensor | me:25632996799 -> alice:67087745702]

In [0]:
%reset -f

## Trusted Aggregator

In [0]:
import torch
import syft as sy
import copy
hook = sy.TorchHook(torch)
from torch import nn, optim

In [0]:
# create a couple workers

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")


# A Toy Dataset
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = torch.tensor([[0],[0],[1],[1.]], requires_grad=True)

# get pointers to training data on each worker by
# sending some training data to bob and alice
bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)

alices_data = data[2:].send(alice)
alices_target = target[2:].send(alice)

In [0]:
# Iniitalize A Toy Model
model = nn.Linear(2,1)

In [0]:
bobs_model = model.copy().send(bob)
alices_model = model.copy().send(alice)

bobs_opt = optim.SGD(params=bobs_model.parameters(),lr=0.1)
alices_opt = optim.SGD(params=alices_model.parameters(),lr=0.1)

In [0]:
for i in range(10):

    # Train Bob's Model
    bobs_opt.zero_grad()
    bobs_pred = bobs_model(bobs_data)
    bobs_loss = ((bobs_pred - bobs_target)**2).sum()
    bobs_loss.backward()

    bobs_opt.step()
    bobs_loss = bobs_loss.get().data

    # Train Alice's Model
    alices_opt.zero_grad()
    alices_pred = alices_model(alices_data)
    alices_loss = ((alices_pred - alices_target)**2).sum()
    alices_loss.backward()

    alices_opt.step()
    alices_loss = alices_loss.get().data
    
    print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))

Bob:tensor(0.2246) Alice:tensor(5.9897)
Bob:tensor(0.0513) Alice:tensor(0.0830)
Bob:tensor(0.0119) Alice:tensor(0.0314)
Bob:tensor(0.0029) Alice:tensor(0.0258)
Bob:tensor(0.0008) Alice:tensor(0.0215)
Bob:tensor(0.0003) Alice:tensor(0.0179)
Bob:tensor(0.0002) Alice:tensor(0.0149)
Bob:tensor(0.0002) Alice:tensor(0.0124)
Bob:tensor(0.0001) Alice:tensor(0.0103)
Bob:tensor(0.0001) Alice:tensor(0.0086)


In [0]:
alices_model.move(secure_worker)

In [0]:
bobs_model.move(secure_worker)

In [0]:
with torch.no_grad():
    model.weight.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
    model.bias.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())

In [0]:
iterations = 10
worker_iters = 5

for a_iter in range(iterations):
    
    bobs_model = model.copy().send(bob)
    alices_model = model.copy().send(alice)

    bobs_opt = optim.SGD(params=bobs_model.parameters(),lr=0.1)
    alices_opt = optim.SGD(params=alices_model.parameters(),lr=0.1)

    for wi in range(worker_iters):

        # Train Bob's Model
        bobs_opt.zero_grad()
        bobs_pred = bobs_model(bobs_data)
        bobs_loss = ((bobs_pred - bobs_target)**2).sum()
        bobs_loss.backward()

        bobs_opt.step()
        bobs_loss = bobs_loss.get().data

        # Train Alice's Model
        alices_opt.zero_grad()
        alices_pred = alices_model(alices_data)
        alices_loss = ((alices_pred - alices_target)**2).sum()
        alices_loss.backward()

        alices_opt.step()
        alices_loss = alices_loss.get().data
    
    alices_model.move(secure_worker)
    bobs_model.move(secure_worker)
    with torch.no_grad():
        model.weight.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
        model.bias.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())
    
    print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))

Bob:tensor(0.0018) Alice:tensor(0.0135)
Bob:tensor(0.0035) Alice:tensor(0.0073)
Bob:tensor(0.0042) Alice:tensor(0.0037)
Bob:tensor(0.0044) Alice:tensor(0.0019)
Bob:tensor(0.0041) Alice:tensor(0.0010)
Bob:tensor(0.0036) Alice:tensor(0.0005)
Bob:tensor(0.0031) Alice:tensor(0.0003)
Bob:tensor(0.0025) Alice:tensor(0.0002)
Bob:tensor(0.0021) Alice:tensor(0.0001)
Bob:tensor(0.0016) Alice:tensor(7.1089e-05)


In [0]:
preds = model(data)
loss = ((preds - target) ** 2).sum()

In [0]:
print(preds)
print(target)
print(loss.data)

tensor([[0.1180],
        [0.0989],
        [0.8747],
        [0.8556]], grad_fn=<AddmmBackward>)
tensor([[0.],
        [0.],
        [1.],
        [1.]], requires_grad=True)
tensor(0.0603)


In [0]:
%reset -f

# Encrypted Programs

In [0]:
Q = 1234567891011

In [0]:
x = 25

In [0]:
import random

def encrypt(x):
    share_a = random.randint(-Q,Q)
    share_b = random.randint(-Q,Q)
    share_c = (x - share_a - share_b) % Q
    return (share_a, share_b,  share_c)

In [0]:
encrypt(x)

(-520169851765, -324163672501, 844333524291)

In [0]:
def decrypt(*shares):
    return sum(shares) % Q

In [0]:
a,b,c = encrypt(25)

In [0]:
decrypt(a, b, c)

25

In [0]:
decrypt(a, b)

728570765909

In [0]:
x = encrypt(25)
y = encrypt(5)

In [0]:
def add(x, y):
    z = list()
    # the first worker adds their shares together
    z.append((x[0] + y[0]) % Q)
    
    # the second worker adds their shares together
    z.append((x[1] + y[1]) % Q)
    
    # the third worker adds their shares together
    z.append((x[2] + y[2]) % Q)
    
    return z

In [0]:
decrypt(*add(x,y))

30

In [0]:
import torch
import syft as sy
hook = sy.TorchHook(torch)

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
bill = sy.VirtualWorker(hook, id="bill")

In [0]:
x = torch.tensor([25])

In [0]:
x

tensor([25])

In [0]:
encrypted_x = x.share(bob, alice, bill)

In [0]:
encrypted_x.get()

tensor([25])

In [0]:
bob._objects

{}

In [0]:
x = torch.tensor([25]).share(bob, alice, bill)

In [0]:
# Bob's share
bobs_share = list(bob._objects.values())[0]
bobs_share

tensor([2025019120486417337])

In [0]:
# Alice's share
alices_share = list(alice._objects.values())[0]
alices_share

tensor([3234094629210624288])

In [0]:
# Bill's share
bills_share = list(bill._objects.values())[0]
bills_share

tensor([3964258287157734208])

In [0]:
Q = x.child.field

(bobs_share + alices_share + bills_share) % Q

tensor([25])

In [0]:
x = torch.tensor([25]).share(bob,alice)
y = torch.tensor([5]).share(bob,alice)

In [0]:
z = x + y
z.get()

tensor([30])

In [0]:
z = x - y
z.get()

tensor([20])

In [0]:
crypto_provider = sy.VirtualWorker(hook, id="crypto_provider")

In [0]:
x = torch.tensor([25]).share(bob,alice, crypto_provider=crypto_provider)
y = torch.tensor([5]).share(bob,alice, crypto_provider=crypto_provider)

In [0]:
# multiplication

z = x * y
z.get()

tensor([125])

In [0]:
x = torch.tensor([[1, 2],[3,4]]).share(bob,alice, crypto_provider=crypto_provider)
y = torch.tensor([[2, 0],[0,2]]).share(bob,alice, crypto_provider=crypto_provider)

In [0]:
# matrix multiplication

z = x.mm(y)
z.get()

tensor([[2, 4],
        [6, 8]])

In [0]:
x = torch.tensor([25]).share(bob,alice, crypto_provider=crypto_provider)
y = torch.tensor([5]).share(bob,alice, crypto_provider=crypto_provider)

In [0]:
z = x > y
z.get()

tensor([1])

In [0]:
z = x <= y
z.get()

tensor([0])

In [0]:
z = x == y
z.get()

tensor([0])

In [0]:
z = x == y + 20
z.get()

tensor([1])

In [0]:
x = torch.tensor([2, 3, 4, 1]).share(bob,alice, crypto_provider=crypto_provider)
x.max().get()

tensor([4])

In [0]:
x = torch.tensor([[2, 3], [4, 1]]).share(bob,alice, crypto_provider=crypto_provider)
max_values, max_ids = x.max(dim=0)
max_values.get()

tensor([4, 3])

In [0]:
%reset -f

# Examples

## Encrypted NN on Encrypted Data

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import syft as sy

In [0]:
# Set everything up
hook = sy.TorchHook(torch) 

alice = sy.VirtualWorker(id="alice", hook=hook)
bob = sy.VirtualWorker(id="bob", hook=hook)
james = sy.VirtualWorker(id="james", hook=hook)

In [0]:
# A Toy Dataset
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]])
target = torch.tensor([[0],[0],[1],[1.]])

# A Toy Model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 2)
        self.fc2 = nn.Linear(2, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x
model = Net()

In [0]:
# We encode everything
data = data.fix_precision().share(bob, alice, crypto_provider=james, requires_grad=True)
target = target.fix_precision().share(bob, alice, crypto_provider=james, requires_grad=True)
model = model.fix_precision().share(bob, alice, crypto_provider=james, requires_grad=True)

In [0]:
print(data)

(Wrapper)>AutogradTensor>FixedPrecisionTensor>[AdditiveSharingTensor]
	-> [PointerTensor | me:94460833089 -> bob:44520530275]
	-> [PointerTensor | me:82868518778 -> alice:16024253777]
	*crypto provider: james*


In [0]:
opt = optim.SGD(params=model.parameters(),lr=0.1).fix_precision()

for iter in range(20):
    # 1) erase previous gradients (if they exist)
    opt.zero_grad()

    # 2) make a prediction
    pred = model(data)

    # 3) calculate how much we missed
    loss = ((pred - target)**2).sum()

    # 4) figure out which weights caused us to miss
    loss.backward()

    # 5) change those weights
    opt.step()

    # 6) print our progress
    print(loss.get().float_precision())

tensor(1.3200)
tensor(1.1790)
tensor(1.1370)
tensor(1.1040)
tensor(1.0800)
tensor(1.0590)
tensor(1.0420)
tensor(1.0300)
tensor(1.0190)
tensor(1.0100)
tensor(1.0010)
tensor(0.9970)
tensor(0.9880)
tensor(0.9820)
tensor(0.9770)
tensor(0.9700)
tensor(0.9630)
tensor(0.9580)
tensor(0.9540)
tensor(0.9440)


In [0]:
%reset -f

## Secure Deep Learning

In [0]:
# Maybe we need to restart the runtime here
epochs = 10
n_test_batches = 200
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
%tensorflow_version 1.x
import tensorflow
from torchvision import datasets, transforms

TensorFlow 1.x selected.


In [0]:
import syft as sy
hook = sy.TorchHook(torch) 
client = sy.VirtualWorker(hook, id="client")
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
crypto_provider = sy.VirtualWorker(hook, id="crypto_provider")

Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.2.so'


In [0]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 50
        self.epochs = epochs
        self.lr = 0.001
        self.log_interval = 100

args = Arguments()

In [0]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.batch_size, shuffle=True)

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


In [0]:
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True)

private_test_loader = []
for data, target in test_loader:
    private_test_loader.append((
        data.fix_precision().share(alice, bob, crypto_provider=crypto_provider),
        target.fix_precision().share(alice, bob, crypto_provider=crypto_provider)
    ))

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

In [0]:
def train(args, model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        output = F.log_softmax(output, dim=1)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(train_loader) * args.batch_size,
                100. * batch_idx / len(train_loader), loss.item()))

In [0]:
model = Net()
optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)

for epoch in range(1, args.epochs + 1):
    train(args, model, train_loader, optimizer, epoch)

Train Epoch: 1 [0/60032 (0%)]	Loss: 2.289751
Train Epoch: 1 [6400/60032 (11%)]	Loss: 0.292684
Train Epoch: 1 [12800/60032 (21%)]	Loss: 0.218543
Train Epoch: 1 [19200/60032 (32%)]	Loss: 0.171335
Train Epoch: 1 [25600/60032 (43%)]	Loss: 0.162887
Train Epoch: 1 [32000/60032 (53%)]	Loss: 0.057719
Train Epoch: 1 [38400/60032 (64%)]	Loss: 0.112598
Train Epoch: 1 [44800/60032 (75%)]	Loss: 0.123485
Train Epoch: 1 [51200/60032 (85%)]	Loss: 0.100780
Train Epoch: 1 [57600/60032 (96%)]	Loss: 0.091225
Train Epoch: 2 [0/60032 (0%)]	Loss: 0.010778
Train Epoch: 2 [6400/60032 (11%)]	Loss: 0.231165
Train Epoch: 2 [12800/60032 (21%)]	Loss: 0.020719
Train Epoch: 2 [19200/60032 (32%)]	Loss: 0.093004
Train Epoch: 2 [25600/60032 (43%)]	Loss: 0.074038
Train Epoch: 2 [32000/60032 (53%)]	Loss: 0.027573
Train Epoch: 2 [38400/60032 (64%)]	Loss: 0.103252
Train Epoch: 2 [44800/60032 (75%)]	Loss: 0.160862
Train Epoch: 2 [51200/60032 (85%)]	Loss: 0.135319
Train Epoch: 2 [57600/60032 (96%)]	Loss: 0.207912
Train Epoch:

In [0]:
def test(args, model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            output = F.log_softmax(output, dim=1)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [0]:
test(args, model, test_loader)


Test set: Average loss: 0.0836, Accuracy: 9817/10000 (98%)



In [0]:
model.fix_precision().share(alice, bob, crypto_provider=crypto_provider)

Net(
  (fc1): Linear(in_features=784, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

In [0]:
def test(args, model, test_loader):
    model.eval()
    n_correct_priv = 0
    n_total = 0
    with torch.no_grad():
        for data, target in test_loader[:n_test_batches]:
            output = model(data)
            pred = output.argmax(dim=1) 
            n_correct_priv += pred.eq(target.view_as(pred)).sum()
            n_total += args.test_batch_size
# This 'test' function performs the encrypted evaluation. The model weights, the data inputs, the prediction and the target used for scoring are all encrypted!

# However as you can observe, the syntax is very similar to normal PyTorch testing! Nice!

# The only thing we decrypt from the server side is the final score at the end of our 200 items batches to verify predictions were on average good.      
            n_correct = n_correct_priv.copy().get().float_precision().long().item()
    
            print('Test set: Accuracy: {}/{} ({:.0f}%)'.format(
                n_correct, n_total,
                100. * n_correct / n_total))

In [0]:
test(args, model, private_test_loader)

Test set: Accuracy: 49/50 (98%)
Test set: Accuracy: 99/100 (99%)
Test set: Accuracy: 148/150 (99%)
Test set: Accuracy: 198/200 (99%)
Test set: Accuracy: 247/250 (99%)
Test set: Accuracy: 297/300 (99%)
Test set: Accuracy: 347/350 (99%)
Test set: Accuracy: 397/400 (99%)
Test set: Accuracy: 446/450 (99%)
Test set: Accuracy: 496/500 (99%)
Test set: Accuracy: 546/550 (99%)
Test set: Accuracy: 592/600 (99%)
Test set: Accuracy: 640/650 (98%)
Test set: Accuracy: 690/700 (99%)
Test set: Accuracy: 739/750 (99%)
Test set: Accuracy: 789/800 (99%)
Test set: Accuracy: 838/850 (99%)
Test set: Accuracy: 886/900 (98%)
Test set: Accuracy: 936/950 (99%)
Test set: Accuracy: 984/1000 (98%)
Test set: Accuracy: 1033/1050 (98%)
Test set: Accuracy: 1081/1100 (98%)
Test set: Accuracy: 1131/1150 (98%)
Test set: Accuracy: 1180/1200 (98%)
Test set: Accuracy: 1229/1250 (98%)
Test set: Accuracy: 1279/1300 (98%)
Test set: Accuracy: 1327/1350 (98%)
Test set: Accuracy: 1376/1400 (98%)
Test set: Accuracy: 1425/1450 (98%